In [22]:

import os
import shutil
import random


In [23]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [24]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git # clone repo
%cd yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [25]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output

In [27]:
%cd /content
!unzip "/content/drive/MyDrive/QR code.v2-qr-code.yolov5pytorch.zip"

Streaming output truncated to the last 5000 lines.
 extracting: train/labels/8283-v3_png.rf.9f93b2b6a64398261ba392996f226489.txt  
 extracting: train/labels/8283-v4_png.rf.b2449520cc5337c4b282ea00e1fb1f60.txt  
 extracting: train/labels/8284-v2_png.rf.ce115cf7ab1790361fe43d2eeb21f01a.txt  
 extracting: train/labels/8284-v3_png.rf.ce61be38bdfa0b508b4c70c885c9f3fe.txt  
 extracting: train/labels/8284-v4_png.rf.70f4ec201315ae881dad955d3c9b14a8.txt  
 extracting: train/labels/8286-v1_png.rf.5a2cb994094c347e8c6f7ff23558b081.txt  
 extracting: train/labels/8286-v2_png.rf.50bb63de0964808b32779d74119cd783.txt  
 extracting: train/labels/8286-v3_png.rf.f3ba8bda38152c283b8dad903bdaf7ea.txt  
 extracting: train/labels/8290-v2_png.rf.8d6d17a1db299bf75dd81d98e0a7dd0c.txt  
 extracting: train/labels/8290-v4_png.rf.a68857be477934073ad0fb71e4c59b24.txt  
 extracting: train/labels/8293-v4_png.rf.8586de5026ae1516614c9b93a1d51a3d.txt  
 extracting: train/labels/8297-v3_png.rf.79ddc966e0648c3afc0eb7878855

In [28]:
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 2
names: ['-', 'QR code']

In [29]:
import yaml
with open('data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

In [30]:
num_classes

'2'

In [31]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]

In [32]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [33]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

nc: {num_classes}
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [34]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 5 --epochs 15 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results  --cache

/content/yolov5
2024-01-12 12:57:59.706615: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 12:57:59.706670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 12:57:59.707983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=./models/custom_yolov5s.yaml, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=5, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device

In [37]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 640 --conf 0.0005 --source ../test1.png

/content/yolov5
detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../test1.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.0005, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-272-gde64179 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/test1.png: 640x640 1 QR code, 11.6ms
Speed: 0.6ms pre-process, 11.6ms inference, 481.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp3


In [44]:
# prompt: save this model to use it further

!python export.py --weights runs/train/yolov5s_results/weights/best.pt --include 'yolov5s.pt' --output 'yolov5s_custom.pt'


usage: export.py [-h] [--data DATA] [--weights WEIGHTS [WEIGHTS ...]] [--imgsz IMGSZ [IMGSZ ...]]
                 [--batch-size BATCH_SIZE] [--device DEVICE] [--half] [--inplace] [--keras]
                 [--optimize] [--int8] [--per-tensor] [--dynamic] [--simplify] [--opset OPSET]
                 [--verbose] [--workspace WORKSPACE] [--nms] [--agnostic-nms]
                 [--topk-per-class TOPK_PER_CLASS] [--topk-all TOPK_ALL] [--iou-thres IOU_THRES]
                 [--conf-thres CONF_THRES] [--include INCLUDE [INCLUDE ...]]
export.py: error: unrecognized arguments: --output yolov5s_custom.pt


In [43]:
# prompt: save the trained model for further reference

!python train.py --img 640 --batch 5 --epochs 15 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results  --cache --save-model


2024-01-12 14:00:21.306178: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 14:00:21.306291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 14:00:21.383455: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP] [--epochs EPOCHS]
                [--batch-size BATCH_SIZE] [--imgsz IMGSZ] [--rect] [--resume [RESUME]] [--nosave]
                [--noval] [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--evolve_population EVOLVE_POPULATION] [--resume_evolve RESUME_EVOLVE]
       

In [45]:
import torch
from models.yolo import Model  # Import YOLOv5 model

# Load your trained YOLOv5 model
model = Model()

# Load the trained weights
model.load_state_dict(torch.load('/content/yolov5/runs/train/yolov5s_results/weights/best.pt'))

# Save the entire model (architecture + weights)
torch.save(model, '/content/yolov5_model.pth')

# If you want to save only the model's state_dict (weights), you can use the following line instead
# torch.save(model.state_dict(), 'path/to/save/your/model/yolov5_weights.pth')


FileNotFoundError: [Errno 2] No such file or directory: 'yolov5s.yaml'